In [1]:
import numpy as np
import sys
import cv2
from math import pow, sqrt

In [2]:
labels = [line.strip() for line in open("class_labels.txt")]

In [4]:
# Load model
network = cv2.dnn.readNetFromCaffe("SSD_MobileNet_prototxt.txt","SSD_MobileNet.caffemodel")

# Capture video from file 
cap = cv2.VideoCapture("vtest.avi")

while cap.isOpened():

    # Capture one frame after another
    ret, frame = cap.read()

    if not ret:
        break

    (h, w) = frame.shape[:2]

    # Resize the frame to suite the model requirements. Resize the frame to 300X300 pixels
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    network.setInput(blob)
    detections = network.forward()

    pos_dict = dict()
    coordinates = dict()
    pair_dict = dict()
    actual_mid = dict()

    # Focal length
    F = 615

    for i in range(detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > 0.2:

            class_id = int(detections[0, 0, i, 1])

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            # Filtering only persons detected in the frame. Class Id of 'person' is 15
            if class_id == 15.00:
                coordinates[i] = (startX, startY, endX, endY)

                # Mid point of bounding box
                x_mid = round((startX+endX)/2,4)
                y_mid = round((startY+endY)/2,4)
                actual_mid[i] = (int(x_mid), int(y_mid))
                
                height = round(endY-startY,4)
                
                # Distance from camera based on triangle similarity
                distance = (165 * F)/height
                print("Distance(cm):{dist}\n".format(dist=distance))
                
                # Mid-point of bounding boxes (in cm) based on triangle similarity technique
                x_mid_cm = (x_mid * distance) / F
                y_mid_cm = (y_mid * distance) / F

                pos_dict[i] = (x_mid_cm,y_mid_cm,distance)

    # Distance between every object detected in a frame
    close_objects = set()
    k=0
    for i in pos_dict.keys():
        for j in pos_dict.keys():
            if i < j:
                dist = sqrt(pow(pos_dict[i][0]-pos_dict[j][0],2) + pow(pos_dict[i][1]-pos_dict[j][1],2) + pow(pos_dict[i][2]-pos_dict[j][2],2))
                
                # Check if distance less than 2 metres or 200 centimetres
                if dist < 200:
                    close_objects.add(i)
                    close_objects.add(j)
                    pair_dict[k] = ( actual_mid[i] ,actual_mid[j] )
                    k+=1
    flag=0
    for i in pos_dict.keys():
        if i in close_objects:
            COLOR = (0,0,255)
            flag = 1
        else:
            COLOR = (0,255,0)
        (startX, startY, endX, endY) = coordinates[i]

        cv2.rectangle(frame, (startX, startY), (endX, endY), COLOR, 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        
    for k in pair_dict.keys():
        cv2.line(frame, pair_dict[k][0], pair_dict[k][1], (0,0,255), 2)
        
    cv2.namedWindow('Frame',cv2.WINDOW_NORMAL)

    # Show frame
    cv2.imshow('Frame', frame)
    cv2.resizeWindow('Frame',800,600)

    key = cv2.waitKey(1) & 0xFF

    # Press `q` to exit
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Distance(cm):970.5882352941177

Distance(cm):1269.2307692307693

Distance(cm):970.5882352941177

Distance(cm):1222.2222222222222

Distance(cm):990.0

Distance(cm):1151.1627906976744

Distance(cm):1000.0

Distance(cm):1151.1627906976744

Distance(cm):1164.7058823529412

Distance(cm):1137.9310344827586

Distance(cm):1100.0

Distance(cm):1285.7142857142858

Distance(cm):1125.0

Distance(cm):1112.3595505617977

Distance(cm):1164.7058823529412

Distance(cm):1010.204081632653

Distance(cm):1112.3595505617977

Distance(cm):1178.5714285714287

Distance(cm):846.1538461538462

Distance(cm):1042.1052631578948

Distance(cm):891.8918918918919

Distance(cm):1137.9310344827586

Distance(cm):1076.0869565217392

Distance(cm):1053.1914893617022

Distance(cm):831.9327731092437

Distance(cm):846.1538461538462

Distance(cm):1356.164383561644

Distance(cm):970.5882352941177

Distance(cm):831.9327731092437

Distance(cm):980.1980198019802

Distance(cm):722.6277372262774

Distance(cm):846.1538461538462

Distan


Distance(cm):1500.0

Distance(cm):1320.0

Distance(cm):1207.3170731707316

Distance(cm):1596.774193548387

Distance(cm):1337.837837837838

Distance(cm):1192.7710843373493

Distance(cm):1596.774193548387

Distance(cm):1337.837837837838

Distance(cm):1650.0

Distance(cm):1112.3595505617977

Distance(cm):1477.6119402985075

Distance(cm):1571.4285714285713

Distance(cm):1302.6315789473683

Distance(cm):1087.9120879120878

Distance(cm):1523.076923076923

Distance(cm):1455.8823529411766

Distance(cm):1337.837837837838

Distance(cm):1100.0

Distance(cm):1414.2857142857142

Distance(cm):1222.2222222222222

Distance(cm):1112.3595505617977

Distance(cm):1434.7826086956522

Distance(cm):1253.1645569620252

Distance(cm):1053.1914893617022

Distance(cm):1269.2307692307693

Distance(cm):1455.8823529411766

Distance(cm):1207.3170731707316

Distance(cm):1434.7826086956522

Distance(cm):1269.2307692307693

Distance(cm):1137.9310344827586

Distance(cm):1375.0

Distance(cm):1253.1645569620252

Distance(

Distance(cm):1151.1627906976744

Distance(cm):1010.204081632653

Distance(cm):1302.6315789473683

Distance(cm):1010.204081632653

Distance(cm):1253.1645569620252

Distance(cm):1622.950819672131

Distance(cm):1394.3661971830986

Distance(cm):980.1980198019802

Distance(cm):1302.6315789473683

Distance(cm):1010.204081632653

Distance(cm):1477.6119402985075

Distance(cm):1020.6185567010309

Distance(cm):1237.5

Distance(cm):1650.0

Distance(cm):1394.3661971830986

Distance(cm):1100.0

Distance(cm):1207.3170731707316

Distance(cm):1100.0

Distance(cm):1622.950819672131

Distance(cm):1394.3661971830986

Distance(cm):1178.5714285714287

Distance(cm):1137.9310344827586

Distance(cm):1192.7710843373493

Distance(cm):1087.9120879120878

Distance(cm):1076.0869565217392

Distance(cm):1125.0

Distance(cm):1192.7710843373493

Distance(cm):1125.0

Distance(cm):1053.1914893617022

Distance(cm):1076.0869565217392

Distance(cm):1571.4285714285713

Distance(cm):1151.1627906976744

Distance(cm):1151.1627

Distance(cm):1414.2857142857142

Distance(cm):1455.8823529411766

Distance(cm):818.1818181818181

Distance(cm):1546.875

Distance(cm):1137.9310344827586

Distance(cm):1650.0

Distance(cm):1477.6119402985075

Distance(cm):678.082191780822

Distance(cm):1414.2857142857142

Distance(cm):1053.1914893617022

Distance(cm):1053.1914893617022

Distance(cm):1477.6119402985075

Distance(cm):1500.0

Distance(cm):738.8059701492538

Distance(cm):733.3333333333334

Distance(cm):1455.8823529411766

Distance(cm):1500.0

Distance(cm):860.8695652173913

Distance(cm):1455.8823529411766

Distance(cm):647.0588235294117

Distance(cm):868.421052631579

Distance(cm):1455.8823529411766

Distance(cm):1546.875

Distance(cm):1137.9310344827586

